In [48]:
# importing necessary packages
import pprint
import pandas as pd
import re

import nltk
from nltk.stem import WordNetLemmatizer

import psycopg2
from sqlalchemy import create_engine


In [52]:
# making a client
client = MongoClient()

# collecting the reviews
ba_database = client['ba_database']
ba_col = ba_database['ba_review_col']


In [58]:
# getting the reviews
reviews = []
for doc in ba_col.find({},{'_id':0, 'review':1}):
        reviews.append(doc['review'])

In [60]:
# downloading stopwords
STOPWORDS = nltk.corpus.stopwords.words('english')

In [61]:
# defining a word lemmatizer
lemmatizer = WordNetLemmatizer()

In [62]:
# function to preprocess the text
def preprocess(text):
        temp_rev = re.sub(r".*\|", "", text.strip())                                    # removing characters before "|"
        temp_rev = re.sub(r"[^\w\s\d]", "", temp_rev)                                   # removing punctuations
        temp_rev = temp_rev.lower()                                                     # converting to lowercase
        temp_rev = temp_rev.split()                                                     # splitting the sentence into words
        temp_rev = [word for word in temp_rev if word not in STOPWORDS]                 # removing stopwords
        temp_rev = [lemmatizer.lemmatize(word) for word in temp_rev]                    # lammtizing the sentence
        
        return temp_rev   

In [63]:
# removing the trip verified from the text and splitting the sentence into individual words
tokens = []
for rev in reviews:
        tokens.append(preprocess(rev))                                          
        


In [73]:
# saving the tokens
for i in range(len(tokens)):
        ba_col.update_one({'_id':i},{'$set':{'tokens':tokens[i]}})